In [227]:
import pandas as pd
import extract_msg
from datetime import datetime
import os
import re

In [228]:
input_folder = r'E:\Telecom Egypt Internship\Python\email_extraction\data'
output_folder = r'E:\Telecom Egypt Internship\Python\email_extraction\output'

In [229]:
columns_names = ["Ticket ID", "Number of Sites", "Site ID", "Related BSC/RNC", "Status", "Alarm Name", "Outage Start", "Outage End", "Operation Area", "Operation Zone", "Assignment", "Subcontractor", "Vendor", "Technology", "System", "Subsystem", "Solution"]

In [230]:
data = pd.DataFrame(columns=columns_names)
data.reset_index(drop=True, inplace=True)

In [231]:
# Iterating over each file we have with .msg extention
for filename in os.listdir(input_folder):
    if filename.endswith('.msg'):
        file_path = os.path.join(input_folder, filename)
        msg = extract_msg.Message(file_path)
        msg_sender = msg.sender
        msg_date = msg.date
        msg_subj = msg.subject
        msg_message = msg.body

    # Extracting the Data we need from the file using Regex
    ticket_id = re.search(r'IM[0-9]*[0-9] | INC-*[0-9]*-*[0-9]*', msg_subj).group()
    sites_num =  re.search("(?s)(?<=Number of Sites).*?(?=Affected NEs)", msg_message).group().strip()
    Related_BSCRNC = re.search("(?s)(?<=Related BSC/RNC).*?(?=Status)", msg_message).group().strip()
    
    if(re.search("Alarm Name", msg_message) != None):
        status = re.search("(?s)(?<=Status).*?(?=Alarm Name)", msg_message).group().strip()
        alarm_name = re.search("(?s)(?<=Alarm Name).*?(?=Outage Start)", msg_message).group().strip()
    else:
        status = re.search("(?s)(?<=Status).*?(?=Outage Start)", msg_message).group().strip()
        alarm_name = " "
    
    if(re.search("Outage End", msg_message) != None):
        outage_start = re.search("(?s)(?<=Outage Start).*?(?=Outage End)", msg_message).group().strip()
        outage_end = re.search("(?s)(?<=Outage End).*?(?=Operation Area)", msg_message).group().strip()
    else:
        outage_start = re.search("(?s)(?<=Outage Start).*?(?=Operation Area)", msg_message).group().strip()
        outage_end = " "
        
    if(re.search("Operation Zone", msg_message) != None):
        operation_area = re.search("(?s)(?<=Operation Area).*?(?=Operation Zone)", msg_message).group().strip()
        operation_zone = re.search("(?s)(?<=Operation Zone).*?(?=Assignment)", msg_message).group().strip()
    else:
        operation_area = re.search("(?s)(?<=Operation Area).*?(?=Assignment)", msg_message).group().strip()
        outage_end = " "
        
    assignment = re.search("(?s)(?<=Assignment).*?(?=Subcontractor)", msg_message).group().strip()

    if(re.search("Vendor", msg_message) != None):
        Subcontractor = re.search("(?s)(?<=Subcontractor).*?(?=Vendor)", msg_message).group().strip()
        Vendor = re.search("(?s)(?<=Vendor).*?(?=Technology)", msg_message).group().strip()
        technology = re.search("(?s)(?<=Technology).*?(?=System)", msg_message).group().strip()
        System = re.findall("(?s)(?<=System).*?(?=Subsystem)", msg_message)[0].strip().split()[-1]
        Subsystem = re.search("(?s)(?<=Subsystem).*?(?=Solution)", msg_message).group().strip()
        Solution = re.search("(?s)(?<=Solution).*?(?=Click)", msg_message).group().strip()
    else:
        Subcontractor = re.search("(?s)(?<=Subcontractor).*?(?=Click)", msg_message).group().strip() 
        Vendor = " "
        technology = " "
        System = " "
        Subsystem = " "
        Solution = " "        

    site_id = re.findall("(?s)(?<=Affected NEs).*?(?=Related BSC/RNC)", msg_message)[0].strip()
    site_ID = site_id.split()
    
    #Adding the Extracted Data to the DataFrame
    for i in range(int(sites_num)):   
        new_record = {"Ticket ID" : ticket_id, "Number of Sites" : sites_num, "Site ID" : site_ID[i], "Related BSC/RNC" : Related_BSCRNC, "Status" : status, "Alarm Name" : alarm_name, "Outage Start" : outage_start, "Outage End" : outage_end, "Operation Area" : operation_area, "Operation Zone" : operation_zone, "Assignment" : assignment, "Subcontractor" : Subcontractor, "Vendor" : Vendor, "Technology" : technology, "System" : System, "Subsystem" : Subsystem, "Solution" : Solution}
        data = data._append(new_record, ignore_index=True)



In [232]:
data.head()

,Ticket ID,Number of Sites,Site ID,Related BSC/RNC,Status,Alarm Name,Outage Start,Outage End,Operation Area,Operation Zone,Assignment,Subcontractor,Vendor,Technology,System,Subsystem,Solution
0,INC-20240811-00001158,1,Orange_Wireless:U_B_2A37AL_SM-MM-MTELALPFRM,Awayed-RNC31,Resolved,NodeB Unavailable,2024-08-11 16:01:54,2024-08-11 19:42:43,Alex,North Coast,Converged Network NC,Orascom,HUAWEI,RAN-3G,Investigation,Under Investigation,
1,IM1647211,26,U_S_UPPER-RS-RD-14_3238UP,RNC62,Open,,02/21/23 09:39:00,,Upper Egypt,North Coast,Converged Network NUP,Huawei,,,,,
2,IM1647211,26,U_S_AT-ELFIMA_3530UP,RNC62,Open,,02/21/23 09:39:00,,Upper Egypt,North Coast,Converged Network NUP,Huawei,,,,,
3,IM1647211,26,U_S_AT-BASRH_3539UP,RNC62,Open,,02/21/23 09:39:00,,Upper Egypt,North Coast,Converged Network NUP,Huawei,,,,,
4,IM1647211,26,U_A_AT-ABNUB-TWN_0102UP,RNC62,Open,,02/21/23 09:39:00,,Upper Egypt,North Coast,Converged Network NUP,Huawei,,,,,


In [233]:
data.to_excel(r"E:\Telecom Egypt Internship\Python\email_extraction\output\Down_Sites.xlsx", sheet_name="Sites", index=False)